<img src = https://i.redd.it/b2dfxek3e9t41.jpg >

The meme above was generated by [GPT2](https://en.wikipedia.org/wiki/GPT-2)

I am a *Get my hands dirty* kind of person. I am also a *Trying the trendiest things out in the market person* . What happens when these two people meet? 
A lovechild. Here's me using GPT2 to write a few Reddit post headlines.
Since I use the [r/VaccineMyths](https://www.kaggle.com/gpreda/reddit-vaccine-myths) Dataset, the output cell contains headlines generated by GPT2.

Enough to fool a human, right?
Let me know in the comments.

<h2> What is Transfer learning? </h2>
In short, Transfer learning is when a model trained for a certain task is reused as a starting point for some other task, saving time and effort of re-training.
This is a helpful resource to read up on:

https://ruder.io/transfer-learning/

<h2> What is GPT-2 </h2>
GPT-2 is a large transformer-based language model with 1.5 billion parameters, trained on a dataset[1]
We created a new dataset which emphasizes diversity of content, by scraping content from the Internet. In order to preserve document quality, we used only pages which have been curated/filtered by humans—specifically, we used outbound links from Reddit which received at least 3 karma. This can be thought of as a heuristic indicator for whether other users found the link interesting (whether educational or funny), leading to higher data quality than other similar datasets, such as CommonCrawl.

[Source](https://openai.com/blog/better-language-models/)

<h1> Imports and installation </h1>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#! pip install deepspeed mpi4py

### Install `aitextgen` package

In [ ]:
!pip install -q aitextgen #install the main package

### Load `aitextgen`

This is the secret sauce

In [ ]:
from aitextgen import aitextgen

### Know your GPU Config 

In [ ]:
! nvidia-smi

### Download the 124M GPT2 Model

In [ ]:
ai = aitextgen(tf_gpt2="124M", to_gpu=True)

### Read the Input Dataset

In [ ]:
input_file = pd.read_csv("../input/reddit-vaccine-myths/reddit_vm.csv")

In [ ]:
pd.set_option('display.max_colwidth', None)

### Text Cleaning (removing comments and keeping only original posts


In [ ]:
input_file.title.value_counts()

In [ ]:
df = pd.DataFrame(input_file["title"])

df=df[df.title!='Comment'].drop_duplicates()

df.shape

### Save the column as text file

In [ ]:
df.to_csv("input_text_cleaned.txt", columns=["title"], header=False, index=False)

In [ ]:
from aitextgen.TokenDataset import TokenDataset

In [ ]:
data = TokenDataset('./input_text_cleaned.txt', line_by_line=True)

### Use the above saved text file for fine-tuning - set the right parameters 

In [ ]:
ai.train('input_text_cleaned.txt',
         line_by_line=False,
         from_cache=False,
         num_steps=500,
         generate_every=100,
         save_every=500,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )

### Load the newly fine-tuned model which is saved in `trained_model` directory

In [ ]:
ai = aitextgen(model_folder="./trained_model/",config="./trained_model/config.json", to_gpu=True)

### Time to see the generated text in action

In [ ]:
ai.generate(n=5,
            batch_size=1,
            max_length=50,
            temperature=1.0,
            top_p=0.9)

<h1> References </h1>

1.https://www.kaggle.com/nulldata/gpt2-generated-indian-food-recipes